In [ ]:
import os
import numpy as np
import rasterio
import math
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from rasterio.transform import rowcol
from joblib import Parallel, delayed


In [ ]:
pasta_principal = '../../Dados/Clima/'

pastas = [nome for nome in os.listdir(pasta_principal) 
          if os.path.isdir(os.path.join(pasta_principal, nome))]
pastas

In [ ]:
def ajustarNumeros(arr):
    return np.where(
            arr >= 100,
            np.round(arr / 10, 2),
            np.round(arr, 2)
        )

In [ ]:
# # Função para salvar o arquivo de saída
def salvarArquivo(dados, profile, caminho, nome):
    os.makedirs(caminho, exist_ok=True)
    saida = os.path.join(caminho, f"{nome}.tif")
    profile.update(dtype='float32', nodata=0)
    with rasterio.open(saida, 'w', **profile) as dst:
        dst.write(dados, 1)

def processarImagens(caminhos, pasta):
    profile = None    
    max_hist = None
    min_hist = None
    soma = None
    print(pasta)
    for caminho in caminhos:
        with rasterio.open(caminho) as src:
            img = ajustarNumeros(src.read(1))
            if profile is None:
                profile = src.profile
              
            if max_hist is None:
                max_hist = img.copy()
                min_hist = img.copy()
                soma = img.astype('float64')
            else:
                max_hist = np.fmax(max_hist, img)
                min_hist = np.fmin(min_hist, img)
                soma += img
                
    avg_hist = soma/len(caminhos)    
    salvarArquivo(max_hist.astype('float32'), profile, pasta, "historico_maximo")
    print(f"Máximo histórico salvo em {pasta}")
  
    salvarArquivo(min_hist.astype('float32'), profile, pasta, "historico_minimo")
    print(f"Mínimo histórico salvo em {pasta}")
  
  
    salvarArquivo(avg_hist.astype('float32'), profile, pasta, "historico_media")
    print(f"Média histórica salvo em {pasta}")
            
            
            
def processarPastas(pastas):
    with ThreadPoolExecutor() as executor:
        futures = []
        for pasta in pastas:
            caminhos = [os.path.join(f"{pasta_principal}/{pasta}", f)
            for f in os.listdir(f"{pasta_principal}/{pasta}") if f.endswith('.tif')]
            futures.append(executor.submit(processarImagens, caminhos, pasta))
        for future in futures:
            future.result()

processarPastas([os.path.join(pasta_principal, pasta) for pasta in pastas])

In [ ]:
with rasterio.open(f"{pasta_principal}/{pastas[1]}/historico_minimo.tif") as src: 
    valores = src.read(1)
    

In [ ]:
df = pd.read_csv("../../Dados/coordenadas_com_indices.csv", index_col=0)
df

In [ ]:
def lerValorPixel(lat, long, imagem):
    with rasterio.open(imagem) as src:
        row, col = rowcol(src.transform, long, lat)
        return src.read(1)[row, col]

In [ ]:
def processarLinha(lat, lon, tiff_path):
    return lerValorPixel(lat, lon, tiff_path)

In [ ]:
tiff_path = f'{pasta_principal}/Temperatura Máxima/historico_maximo.tif'

df['TempMax-Max'] = Parallel(n_jobs=4)(
    delayed(processarLinha)(row.Latitude, row.Longitude, tiff_path) for row in tqdm(df.itertuples(index=False), total=len(df), desc="Processando")
)